Написать программу для построения обратного индекса по собранным текстам. Необходимо реализовать возможность поиска по нескольким словам.

In [1]:
import os
import pickle
import json
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from collections import defaultdict


def tokenize_text(text):
    tokens = nltk.word_tokenize(text.lower())
    clean_tokens = [
        word
        for word in tokens
        if word.isalnum() and word not in stopwords.words("russian")
    ]
    return clean_tokens


def build_inverted_index(texts):
    inverted_index = defaultdict(list)

    for doc_id, text in tqdm(
        enumerate(texts), desc="Building inverted index", total=len(texts)
    ):
        if text is None:
            continue
        tokens = tokenize_text(text)
        for token in set(tokens):
            inverted_index[token].append(doc_id)
    return inverted_index


def load_inverted_index(path, texts):
    if os.path.exists(path):
        with open(path, "rb") as f:
            return pickle.load(f)
    else:
        result = build_inverted_index(texts)
        with open(path, "wb") as f:
            pickle.dump(result, f)


def search(query, inverted_index):
    query_tokens = set(tokenize_text(query))
    search_results = set()

    for token in query_tokens:
        if token in inverted_index:
            if not search_results:
                search_results = set(inverted_index[token])
            else:
                search_results = search_results.intersection(set(inverted_index[token]))

    return search_results

In [2]:
with open("../naked_science_corpus.json", "r") as f:
    data = json.load(f)

texts = [article["text"] for article in data]

inverted_index = load_inverted_index("inverted_index.pickle", texts)

In [12]:
query = "искуственный интеллект"
results = search(query, inverted_index)

print('Результаты поиска для запроса "{}":'.format(query))
for result in results:
    print(result, texts[result])

Результаты поиска для запроса "искуственный интеллект":
131 Представители Министерства здравоохранения России представили достижения по шести направлениям: генетические и нейротехнологии, регенеративная и ядерная медицина, биобезопасность, цифровые технологии и искусственный интеллект. «Выставочная экспозиция Форума будущих технологий представляет прорывные технологии медицины и передовые достижения нашего научного сообщества. Многие прототипы, представленные сегодня, уже готовы к внедрению, ряд проектов реализуется на территории страны. Инновации внедряются постоянно, и то, что вчера мы назвали нашим будущим, сегодня уже реализовано и успешно применяется в повседневной жизни», — отметила заместитель председателя правительства России, сопредседатель организационного комитета Татьяна Голикова. Роспотребнадзор продемонстрировал технологии, направленные на обеспечение биобезопасности: мобильные комплексы, возможности современной лабораторной диагностики, платформы VGARus и SOLA, технологи

In [13]:
query = "ядерная физика"
results = search(query, inverted_index)

print('Результаты поиска для запроса "{}":'.format(query))
for result in results:
    print(result, texts[result])

Результаты поиска для запроса "ядерная физика":
2691 Британская версия событий: операция «Новичок». На Западе популярны целые книги, описывающие, как именно действия союзников лишили Третий рейх шансов на ядерную бомбу. Ключевая линия повествования там такая: для получения бомбы Гитлеру нужна была тяжелая вода (такая, где вместо обычного водорода тяжелый — дейтерий). Восемнадцатого октября 1942 года англичане сбросили четырех диверсантов передовой группы в районе норвежской фабрики по производству тяжелой воды (операция «Шотландская куропатка», традиционно ошибочно переводится как «Тетерев»). Туда же 19 ноября того же года на планерах должны были высадиться 34 диверсанта-англичанина из SAS (операция «Новичок», традиционно ошибочно переводится как «Незнакомец»). Два четырехмоторных самолета-буксира тащили два планера Airspeed AS.51 Horsa, где и размещались десантники. Один буксир потерял планер (обрыв троса) не там, где нужно было высадиться. Трое сасовцев погибли, остальных 14 немцы вс